In [1]:
import numpy as np
#from modules import AttentionBlock
import torch
import os

from PIL import Image
import matplotlib.pyplot as plt
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
import glob
from stage_1_models import AttentionBlock, Stage1, DiceLoss
from torchsummary import summary
import pickle
device = torch.device("cpu")

if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available :(")
  device = torch.device("cpu")

Cuda (GPU support) is available and enabled!


In [2]:
resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')

resnet50.eval().to(device)
resnet50 = resnet50.to(device)

Using cache found in /home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
Using cache found in /home/umut/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [3]:
class Stage1Dataset(Dataset):
  def __init__(self, train_folder):

    self.dirs = []
    self.lengths = []
    self.N = 0

    for dir in os.listdir(train_folder):
      # take the subfolders
      self.dirs.append(os.path.join(train_folder, dir))
      self.lengths.append(len(os.listdir(os.path.join(train_folder, dir) + "/rgb")))
      self.N += self.lengths[-1]


  # Find the correct folder and file name given index
  def find_index(self, idx):
    temp = idx
    for i in range(len(self.dirs)):
      if temp < self.lengths[i]:
        return self.dirs[i], temp
      temp -= self.lengths[i]


  def __len__(self):
    return self.N

  def __getitem__(self, idx):
    fold, file = self.find_index(idx)

    img = Image.open(os.path.join(fold, "rgb") + "/" + str(file).zfill(4) + ".png").resize((480,640))
    convert_tensor = transforms.Compose([transforms.ToTensor(), transforms.Resize((480, 640))])

    image = convert_tensor(img)
    return image

In [4]:
rgb_folder = "/home/umut/Desktop/501/dataset/T-LESS/t-less_v2/train_primesense/"

dataset = Stage1Dataset(rgb_folder)

In [5]:
nodes, _ = get_graph_node_names(resnet50)

return_nodes = {
    'layers.0.0.downsample.1': 'layer1',
    'layers.1.0.downsample.1': 'layer2',
    'layers.2.0.downsample.1': 'layer3'
}

extractor = create_feature_extractor(resnet50, return_nodes=return_nodes)

In [6]:
def get_spatial_average(val):
    print(val.shape)
    val = torch.mean(val, dim=3)
    val = torch.mean(val, dim=2)
    diff = torch.mean(val, dim=1)

    res = val - diff

    return res

In [ ]:
features_dict = None
with open('data/features.pkl', 'rb') as f:
    features_dict = pickle.load(f)
features = {}
features["o1"] = features_dict["layer1"]
features["o2"] = features_dict["layer2"]
features["o3"] = features_dict["layer3"]

model = Stage1(features, extractor, resnet50)
model = model.to(device)
summary(model, input_size=[(256,120,160), (512, 60, 80), (1024, 30, 40)], device="cuda")

In [22]:
# Your implementation comes here
def train(model, criterion, optimizer, epochs, dataloader, verbose=True):
  """
    Define the trainer function. We can use this for training any model.
    The parameter names are self-explanatory.

    Returns: the loss history.
  """
  loss_history = []

  for epoch in range(epochs):
    for i, data in enumerate(dataloader, 0):    
      
      # Our batch:
      inputs, labels = data
      inputs = inputs.to(device)
      labels = labels.to(device)

      # zero the gradients as PyTorch accumulates them
      optimizer.zero_grad()
      input_features = model.extractor(inputs)
        
      # Obtain the scores
      outputs = model(input_features["layer1"], input_features["layer2"], input_features["layer3"])

      # Calculate loss
      loss = criterion(outputs.to(device), labels)

      # Backpropagate
      loss.backward()

      # Update the weights
      optimizer.step()

      loss_history.append(loss.item())
    if epoch % 100 == 0:
        torch.save(model, model_path + "train_" + str(epoch) + ".pth")
        with open(model_path + "hist_" + str(epoch) + ".txt", "w") as f:
            f.write(str(loss_history))
    if verbose: 
      print(f'Epoch {epoch} / {epochs}: avg. loss of last 5 iterations {np.sum(loss_history[:-6:-1])/5}')

  return model, loss_history

In [ ]:
loss = DiceLoss()
model = Stage1(features, extractor, resnet50)
optim = torch.optim.Adam(model.parameters(), lr=2e-4)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True, num_workers=10)
train_model = True

if train_model:
    model, loss_history = train(model, loss, optim, 100, dataloader)
else:
    model = torch.load(model_path + "train.pth")